In [1]:
import pandas as pd
import pickle
import numpy as np
from scipy.stats import norm


# here we set up all columns
pd.set_option('max_columns', 64)
pd.set_option('max_rows',64)

In [2]:
cust_sev.shape

NameError: name 'cust_sev' is not defined

In [ ]:
cust_sev['cancel_date'].isna().sum()

In [ ]:
cust_sev.columns

# customer_service_reps

In [3]:
datafile = open('customer_service_reps', 'rb')

cust_sev = pickle.load(datafile)

datafile.close()

In [ ]:
cust_sev.columns

In [ ]:
cust_sev.shape

In [ ]:
# There are 1369360 users in total in the customer_service_reps data
cust_sev['subid'].nunique()

In [ ]:
cust_sev['billing_channel'].unique()

## OTT Channel

There are three billing channels \
For <b>OTT channel</b>, there are only 2 trial days, <b>14 and 0</b> which we can use for <b>AB_Testing</b>

In [4]:
cs_ott = cust_sev[ cust_sev['billing_channel'] == 'OTT'].reset_index(drop=True)

In [5]:
cs_ott['num_trial_days'].unique()

array([14,  0])

### Preprocessing the data

In [6]:
cs_ott['renew'].unique()
# as you may see here, there are nan values that we need to fill in by ourselves

array([False, True, nan], dtype=object)

In [7]:
cs_ott['renew'].value_counts()

False    387666
True     218581
Name: renew, dtype: int64

In [8]:
cs_ott.index

RangeIndex(start=0, stop=1848663, step=1)

In [9]:
cs_ott['last_payment'].dtype

dtype('<M8[ns]')

In [10]:
cs_ott.tail(50)

,customer_service_rep_id,subid,current_sub_TF,cancel_date,account_creation_date,num_trial_days,trial_completed_TF,billing_channel,revenue_net_1month,payment_period,last_payment,next_payment,renew
1848613,34246054,24911314,False,NaT,2019-06-30 00:02:00,14,True,OTT,4.5875,0,2019-06-30 00:02:00,2019-07-14 10:43:57,True
1848614,34246054,24911314,False,NaT,2019-06-30 00:02:00,14,True,OTT,4.5875,1,2019-07-14 10:43:57,NaT,False
1848615,31230756,27609410,True,NaT,2019-06-30 00:01:58,14,True,OTT,5.8353,0,2019-06-30 00:01:58,2019-07-14 18:43:52,NaN
1848616,31230756,27609410,True,NaT,2019-06-30 00:01:58,14,True,OTT,5.8353,1,2019-07-14 18:43:52,2020-03-13 19:27:38,NaN
1848617,31230756,27609410,True,NaT,2019-06-30 00:01:58,14,True,OTT,5.8353,2,2020-03-13 19:27:38,NaT,NaN
1848618,36080216,27196054,True,NaT,2019-06-30 00:01:56,14,True,OTT,4.5875,0,2019-06-30 00:01:56,2019-07-14 14:34:33,True
1848619,36080216,27196054,True,NaT,2019-06-30 00:01:56,14,True,OTT,4.5875,1,2019-07-14 14:34:33,2019-11-11 13:36:12,NaN
1848620,36080216,27196054,True,NaT,2019-06-30 00:01:56,14,True,OTT,4.5875,2,2019-11-11 13:36:12,2020-03-13 14:01:42,NaN
1848621,36080216,27196054,True,NaT,2019-06-30 00:01:56,14,True,OTT,4.5875,3,2020-03-13 14:01:42,NaT,False
1848622,34685812,23179152,False,2019-07-10 21:09:46,2019-06-30 00:01:53,14,False,OTT,0.0000,0,2019-06-30 01:05:31,NaT,NaN


In [11]:
def renew_filling(period, nxt):
    if period == 0:
        if str( nxt ) == 'NaT':
            return(False)
        else:
            return(True)
            

In [12]:
# we apply the function calc_win_perc to baseball_df, where row is the iterator
# here the run_diffs_apply we created is pandas series
run_diffs_apply = baseball_df.apply(
lambda row: calc_win_perc(row['W'], row['G']),
axis = 1
)
baseball_df["WP_3"] = run_diffs_apply

NameError: name 'baseball_df' is not defined

In [13]:
def calc_win_perc(wins, games_played):
    
    win_perc = wins / games_played
    
    return(np.round(win_perc,2))
    

In [14]:
cs_ott['renew'] = cs_ott.apply(lambda row: renew_filling( row['payment_period'], row['next_payment'] ), axis = 1)

In [15]:
cs_ott.head(20)

,customer_service_rep_id,subid,current_sub_TF,cancel_date,account_creation_date,num_trial_days,trial_completed_TF,billing_channel,revenue_net_1month,payment_period,last_payment,next_payment,renew
0,33513851,28948735,False,2020-04-06 19:21:05,2020-03-27 23:59:39,14,False,OTT,0.0000,0,2020-03-27 23:59:39,NaT,False
1,30034436,21697364,False,2020-04-08 13:16:39,2020-03-27 23:59:06,14,False,OTT,0.0000,0,2020-03-28 00:05:20,NaT,False
2,31856201,27800927,True,NaT,2020-03-27 23:59:04,14,True,OTT,4.5875,0,2020-03-28 00:05:20,2020-04-11 11:05:24,True
3,31856201,27800927,True,NaT,2020-03-27 23:59:04,14,True,OTT,4.5875,1,2020-04-11 11:05:24,NaT,None
4,34244883,26594255,False,2020-04-02 23:06:20,2020-03-27 23:59:00,14,False,OTT,0.0000,0,2020-03-28 08:35:56,NaT,False
5,34138852,20407871,False,2020-04-01 05:42:28,2020-03-27 23:58:49,14,False,OTT,0.0000,0,2020-03-27 23:58:49,NaT,False
6,34428685,28893238,True,NaT,2020-03-27 23:58:43,14,True,OTT,5.6885,0,2020-03-28 07:10:03,2020-04-11 14:22:46,True
7,34428685,28893238,True,NaT,2020-03-27 23:58:43,14,True,OTT,5.6885,1,2020-04-11 14:22:46,NaT,None
8,30133343,29755063,False,2020-03-28 00:04:30,2020-03-27 23:58:43,14,False,OTT,0.0000,0,2020-03-28 07:10:03,NaT,False
9,36265602,25208989,False,2020-04-11 01:42:12,2020-03-27 23:58:34,14,False,OTT,0.0000,0,2020-03-28 07:10:03,NaT,False


In [16]:
cs_ott_14 = cs_ott[np.logical_and( cs_ott['num_trial_days'] == 14 , cs_ott['payment_period'] == 0 )].reset_index(drop=True)

In [17]:
cs_ott_0 = cs_ott[np.logical_and( cs_ott['num_trial_days'] == 0 , cs_ott['payment_period'] == 0 )].reset_index(drop=True)

In [18]:
convt_cs_ott_14 = cs_ott_14['renew'].sum() / cs_ott_14['renew'].count()
convt_cs_ott_14

0.40613574657624213

In [19]:
cs_ott_14['renew'].count()

1202918

In [20]:
convt_cs_ott_0 = cs_ott_0['renew'].sum() / cs_ott_0['renew'].count()
convt_cs_ott_0

0.7578372159907967

In [21]:
alpha = 0.1

z_stats = (convt_cs_ott_14-convt_cs_ott_0)/np.sqrt(convt_cs_ott_0*(1-convt_cs_ott_0)/ cs_ott_14['renew'].count() )
z_alpha = norm.ppf(1-alpha/2)

if z_stats<z_alpha:
    print("Accept Null Hypothesis")
else:
    print("Reject Null Hypothesis")

Accept Null Hypothesis


In [22]:
z_stats

-900.4311892645993

In [23]:
z_alpha 

1.6448536269514722

### Google


In [ ]:
cs_o_7 = cust_sev[ np.logical_and( cust_sev['billing_channel'] == 'OTT', cust_sev['num_trial_days'] == 14 ) ]

In [ ]:
cs_o_7.head(20)

In [ ]:
cust_sev[cust_sev['subid']==27800927][['subid','num_trial_days','payment_period','billing_channel','revenue_net_1month','last_payment', 'next_payment', 'renew']]

In [ ]:
cust_sev[cust_sev['subid']==25914865][['subid','num_trial_days','payment_period','billing_channel','revenue_net_1month','last_payment', 'next_payment', 'renew']]

In [ ]:
subscribers[subscribers['subid'] == 25914865]

In [ ]:
cust_sev.reset_index().head(10)

In [ ]:
datafile = open('engagement', 'rb')

engagement = pickle.load(datafile)

datafile.close()

In [ ]:
eng_1 = engagement[engagement['date'] == '2019-07-01']

# descriptive analysis
in engagement_1 file, there are 4227 unique users

In [ ]:
eng_1.sort_values(['subid','date'], ascending = True)['subid'].nunique()

In [ ]:
eng_1['subid'].nunique()

In [ ]:
list(engagement['payment_period'].unique())

In [ ]:
engagement.reset_index().head(5)

In [ ]:
datafile = open('subscribers', 'rb')

subscribers = pickle.load(datafile)

datafile.close()

In [ ]:
subscribers.shape

In [ ]:
subscribers.columns

In [ ]:
subscribers[['country','plan_type']]

In [ ]:
subscribers['plan_type'].unique()

# descriptive analysis
in //subscriber// there is 227628 users

In [ ]:
subscribers['subid'].nunique()

In [ ]:
subscribers.reset_index()

In [ ]:
cust_sev['subid'].nunique()

In [ ]:
cust_sev.columns = ['customer_service_rep_id', 'user_id', 'current_sub_TF', 'cancel_date',
       'account_creation_date', 'num_trial_days', 'trial_completed_TF',
       'billing_channel', 'revenue_net_1month', 'payment_period',
       'last_payment', 'next_payment', 'renew']

In [ ]:
combined = pd.merge(subscribers, cust_sev, left_on = 'subid', right_on = 'user_id', how = 'left')

In [ ]:
combined['']

In [ ]:
cust_sev[cust_sev['subid'] == 21724479]